In [2]:
import numpy as np
import pandas as pd
import re
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem.lancaster import LancasterStemmer

In [3]:
# import nltk
# nltk.download('stopwords')

In [4]:
matrix = pd.read_parquet('../datasets/processed/frequency_matrix_2013.parquet')

In [5]:
def remove_stopwords(text: str) -> str:
    stop_words = set(stopwords.words('portuguese'))
    new_text = [item for item in wordpunct_tokenize(text) if item not in stop_words]
    return " ".join(new_text)

In [6]:
def lc_stem(text: str) -> str:
    stemmer = LancasterStemmer()
    words = [stemmer.stem(word) for word in wordpunct_tokenize(text)]
    return " ".join(words)

In [7]:
resumo = 'Este trabalho foi desenvolvido utilizando as tecnologias Python, Pandas, Numpy, NLTK, Unidecode e LancasterStemmer. O objetivo é realizar a limpeza de um texto, removendo stopwords, pontuações e acentuações, além de realizar a redução de palavras para sua forma raiz. O texto utilizado para teste foi o resumo deste trabalho. nitrosilo complexo rutenio'

In [8]:
resumo = resumo.lower()

In [9]:
resumo = re.sub(r'[^a-zA-Z0-9\s]+', '', resumo)

In [10]:
resumo = remove_stopwords(resumo)
resumo = lc_stem(resumo)
resumo = unidecode(resumo)

del remove_stopwords, lc_stem

In [11]:
resumo

'trabalho desenvolvido utilizando tecnologia python panda numpy nltk unidecod lancasterstem objetivo realiz limpez texto removendo stopword pontua acentua alm realiz reduo palavra form raiz texto utilizado test resumo dest trabalho nitrosilo complexo rutenio'

In [12]:
def verify_concat(i: int, item: str, text: str) -> str:
    elements = wordpunct_tokenize(text)[i:]
    
    if len(elements) > 1:
        test_1 = item + ' ' + elements[1]
        test_2 = item  + elements[1]
    else:
        return i, item
    
    if test_1 in matrix.palavra_chave.values:
        return verify_concat(i+1, test_1, text)
    if test_2 in matrix.palavra_chave.values:
        return verify_concat(i+1, test_2, text)
    else:
        return i, item

def get_keywords(text: str) -> list:
    result = []
    i = 0
    elements = wordpunct_tokenize(text)
    while i < len(elements):
        item = elements[i]
        if item in matrix.palavra_chave.values:
            i, item = verify_concat(i, item, text)
            result.append(item)
        i+=1
    return result

In [25]:
result = list(set(get_keywords(resumo)))
result

['texto',
 'utilizando',
 'nltk',
 'test',
 'nitrosilo complexo rutenio',
 'limpez',
 'palavra',
 'raiz',
 'python',
 'objetivo',
 'alm',
 'tecnologia',
 'resumo',
 'form',
 'trabalho']

In [27]:
# matrix[matrix.palavra_chave.str.fullmatch(result[0])].sort_values(by='frequencia', ascending=False)
matrix[matrix.palavra_chave.str.fullmatch(result[0])].frequencia.sum()

20023

In [29]:
frequency_general = matrix.frequencia.sum()
for item in result:
    this_frequency = matrix[matrix.palavra_chave.str.fullmatch(item)].frequencia.sum()
    print(f'{item}: {this_frequency/frequency_general}')

texto: 0.0004344467049019401
utilizando: 0.0002659665239318774
nltk: 4.3394766508708993e-08
test: 0.0006401812929197295
nitrosilo complexo rutenio: 3.6885551532402644e-07
limpez: 1.0544928261616285e-05
palavra: 6.162056844236677e-05
raiz: 2.759907149953892e-05
python: 3.4715813206967195e-07
objetivo: 0.0009714786378304682
alm: 0.000552567259338646
tecnologia: 7.871810644679811e-05
resumo: 2.7946229631608592e-05
form: 0.0023989277847511962
trabalho: 0.0012835737985611034


In [34]:
subarea_groupped = matrix.groupby('subarea').frequencia.sum().sort_values(ascending=False).reset_index()

In [56]:
for item in result:
    words_frequency = matrix[matrix.palavra_chave.str.fullmatch(item)].sort_values(by='frequencia', ascending=False).loc[:, ['subarea', 'frequencia']].copy()
    matrix_corr = words_frequency.join(subarea_groupped.set_index('subarea'), on='subarea', how='left', lsuffix='_this_word', rsuffix='_all_groupped')
    matrix_corr['corr'] = matrix_corr.frequencia_this_word / matrix_corr.frequencia_all_groupped
    corr = matrix_corr.sort_values(by='corr', ascending=False).reset_index(drop=True).loc[0, ['subarea', 'corr']]
    print(f'{item}: {corr[0]} - {corr[1]}', end='\n\n')

texto: linguística e literatura - 0.0022056152369090247

utilizando: química - 0.0007176777644514405

nltk: linguística e literatura - 1.0591189613008524e-06

test: educação física - 0.002297859121959878

nitrosilo complexo rutenio: ciências ambientais - 5.564107795315021e-06

limpez: geografia - 7.642830421683112e-05

palavra: linguística e literatura - 0.0002022917216084628

raiz: ciências agrárias i - 0.00019929185401145797

python: ciência da computação - 4.810994083680026e-06

objetivo: educação física - 0.0013770513627677484

alm: ciências biológicas ii - 0.0009491128115669826

tecnologia: ensino - 0.00035947116656096504

resumo: zootecnia / recursos pesqueiros - 0.00013288488041870818

form: comunicação e informação - 0.00538162865537988

trabalho: matemática / probabilidade e estatística - 0.0034520993163137505

